<center><img src="./chunking.png" width="200"></center>

# CV Chunking PDF Docuemnts, Load in dataabse Prompt RAG question 
sono configurati diversi metodi di chunking riferiemnto 
https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a

* Spacy Sentence Splitter 
* Langchain Character Text Splitter  
* NLTK Sentence Tokenizer 

Pacchetti da installare 

In [ ]:
pip install PyPDF2 nltk pydantic langchain langchain==0.0.349 spacy oracledb sentence_transformers sklearn.cluster python-dotenv time

In [ ]:
!python3 -m spacy download en_core_web_sm

In [ ]:
pip install ./drivers/oracledb-2.0.0.dev20231121-cp311-cp311-macosx_10_9_universal2.whl

Fine pacchetti da installare

Cartella con pdf dei CV


In [17]:
folder = './docs/CV_PDF'

Variabili di ambiente 
creare un file .env in questa cartella seguendo il formato TOML con le variabili e salvarlo 
CO_API_KEY=""
USERNAME=""
PASSWORD=""
HOST=""
PORT=""
SERVICE_NAME=""


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

# leggi il valore delle variabile d'ambiente 
cohere_api_key =os.environ.get('CO_API_KEY') #keep your cohere api key  port  in the .env file and retrieve it
oracle_service_name =os.environ.get('SERVICE_NAME') #keep your oracle listner port  in the .env file and retrieve it
oracle_user = os.environ.get('USERNAME') #keep your oracle username in the .env file and retrieve it
oracle_password =os.environ.get('PASSWORD') #keep your oracle usernamepassword in the .env file and retrieve it
oracle_hostname =os.environ.get('HOST') #keep your oracle server hostname in the .env file and retrieve it
oracle_port =os.environ.get('PORT') #keep your oracle listner port  in the .env file and retrieve it

#verifica
#print (oracle_service_name,cohere_api_key)

Cohere Api key 

In [3]:
import cohere
co = cohere.Client(cohere_api_key)

Oracle enviroment 

In [4]:
import oracledb
# Connect to your Oracle 23.4 database
#print (oracle_user, oracle_password,oracle_hostname,oracle_port,oracle_service_name)
connection = oracledb.connect(
    user=oracle_user,
    password=oracle_password,
    dsn=oracle_hostname+":"+oracle_port+"/"+oracle_service_name
    )
cursor = connection.cursor()


Drop and create table Vector_CV_PDF

In [5]:
cursor.execute("""
    begin
        execute immediate 'drop table Vector_CV_PDF';
        exception when others then if sqlcode <> -942 then raise; end if;
    end;""")
# Create table Vector_TEST1
cursor.execute("""
    create table Vector_CV_PDF (
        id_pdf number,
        id_chunk number,
        nome_file varchar2(300),       
        v vector(1024, float32),
        chunk varchar2(4000),
        primary key (id_pdf,id_chunk))""")

#v vector(384, float32) --> 384 e' la dimensione

Funzione per estrarre i file PDF  all'interno della cartella fornita

In [6]:
def elenca_files_con_percorsi(carta):
    lista_files = []
    for cartella_radice, sottocartelle, files in os.walk(carta):
        for file in files:
            if file.lower().endswith('.pdf'):  # Verifica se il file ha estensione PDF
                  percorso_completo = os.path.join(cartella_radice, file)
                  lista_files.append(percorso_completo)
    return lista_files
#print (len(lista_file))
#print (lista_file)


Funzione per convertire pdf to testo

In [11]:
from PyPDF2 import PdfReader
# Extracting Text from PDF
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = " ".join(page.extract_text() for page in pdf.pages)
        
    return text



# Spacy
Funzione per suddivisione in chunk mediante spacy del testo , tennde a creare piccoli chiuck in confronto con  Langchain Character Text Splitter, . Puo essere vantaggioso con piccoli testi 

In [19]:
import spacy
# chuk Text using spacy 
def divide_in_frase(testo):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(testo)
    frasi = list(doc.sents)
    return frasi
#testo_da_dividere = "Questo è un esempio di testo. Può contenere più di una frase. SpaCy è uno strumento di elaborazione del linguaggio naturale."
#frasi_divise = divide_in_frase(testo_da_dividere)


# Langchain Character Text Splitter
RecursiveCharacterTextSplitter ricorsivamente divide il testo sulla base di specifici caratteri .
Puo essere vantaggioso con Testi generici

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def custom_text_splitter(text, chunk_size=100, chunk_overlap=10):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    chunks = splitter.create_documents([text])
    return chunks




# NLTK Sentence Tokenizer
Il Natural Language Toolkit (NLTK) fornisce una funzione utile per suddividere il testo in frasi. e' un modo semplice ed efficiente per dividere un ampio corpo di testo in singole frasi,ma ha alcune limitazioni:
* Dipendenza dalla lingua:Funziona bene con l'inglese ma potrebbe non fornire risultati accurati con altre lingue senza una configurazione aggiuntiva.
* Abbreviazioni e punteggiatura: il tokenizzatore può occasionalmente interpretare erroneamente le abbreviazioni o altri segni di punteggiatura.
* Mancanza di comprensione semantica: come la maggior parte dei tokenizzatori, il tokenizzatore di frasi NLTK non considera la relazione semantica tra le frasi. Pertanto, un contesto che si estende su più frasi potrebbe andare perso nel processo di tokenizzazione.

* nltk.punkt è un modulo di NLTK (Natural Language Toolkit) che fornisce un tokenizer di frasi. Il tokenizer divide un testo in una lista di frasi utilizzando un algoritmo non supervisionato per costruire un modello per le parole di abbreviazione, le collocazioni e le parole che iniziano le frasi.
* nltk.corpus: fornisce accesso a una vasta gamma di corpora etichettati e non etichettati, come il corpus Brown, il corpus Penn Treebank, il corpus WordNet e molti altri.
* nltk.tokenize: fornisce una serie di tokenizzatori per la suddivisione del testo in parole, frasi e altre unità.
* nltk.stem: fornisce una serie di algoritmi di stemming per ridurre le parole alle loro forme di base.
* nltk.tag: fornisce una serie di modelli di tagging per l’etichettatura delle parti del discorso, l’etichettatura delle entità denominate e altro ancora.
* nltk.chunk: fornisce una serie di modelli di chunking per l’identificazione di frasi nominali, verbi e altre unità sintattiche.
* nltk.parse: fornisce una serie di parser per l’analisi sintattica del testo.
* nltk.sentiment: fornisce una serie di modelli per l’analisi del sentiment.

In [ ]:
import nltk
nltk.download('punkt')

def split_text_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

# Confronta i diversi metodi Langchain

In [22]:
id_file=0
lista_file= elenca_files_con_percorsi(folder)
for nume_file in range(len(lista_file)):
    id_file+=1
    id_chunk=0
    # print (id_nome_file)
    # Extract text from the PDF and split it into sentences
    nome_file=lista_file[nume_file]
    text = extract_text_from_pdf(lista_file[nume_file])
    #print (text)
    spacy_frasi_divise = divide_in_frase(text)
    splitter_result_chunks = custom_text_splitter(text)
    nltk_sentences = split_text_into_sentences(text)
    print ('file : ', nume_file)
    print ('Spacy Sentence Splitter num chunk ',len(spacy_frasi_divise))
    print ('* Langchain Character Text Splitter num chunk ',len(splitter_result_chunks))
    print ('NLTK Sentence Tokenizer num chunk ',len(nltk_sentences))

   # print (spacy_frasi_divise)
   # print (splitter_result_chunks)
   # print (splitter_result_chunks[0].page_content)
   # print (nltk_sentences)


file :  0
Spacy Sentence Splitter num chunk  17
* Langchain Character Text Splitter num chunk  15
NLTK Sentence Tokenizer num chunk  15
file :  1
Spacy Sentence Splitter num chunk  19
* Langchain Character Text Splitter num chunk  22
NLTK Sentence Tokenizer num chunk  16
file :  2
Spacy Sentence Splitter num chunk  15
* Langchain Character Text Splitter num chunk  28
NLTK Sentence Tokenizer num chunk  14
file :  3
Spacy Sentence Splitter num chunk  19
* Langchain Character Text Splitter num chunk  22
NLTK Sentence Tokenizer num chunk  16
file :  4
Spacy Sentence Splitter num chunk  14
* Langchain Character Text Splitter num chunk  24
NLTK Sentence Tokenizer num chunk  13


 # Implementazione
 Conversione  di tutti i pdf, della cartella fornita, in text, successivamente suddiviso il testo  in chuck mediante spacy e traformati in vector embedding con Cohere embed , alla fine inseriti all'interno della tabella vector_PDF del database Oracel

In [25]:
import time
id_file=0

#definizione type per oracle variabili
cursor.setinputsizes(None,None,oracledb.DB_TYPE_VARCHAR, oracledb.DB_TYPE_VECTOR )

#definizione del modello utilizzato da cohere e il metodo per l'embedding orientato allo store degli stessi
cohere_model_def='embed-english-v3.0'
cohere_input_type='search_document'

lista_file= elenca_files_con_percorsi(folder)
for nume_file in range(len(lista_file)):
    id_file+=1
    id_chunk=0
    # print (id_nome_file)
    # Extract text from the PDF and split it into sentences
    nome_file=lista_file[nume_file]
    text = extract_text_from_pdf(lista_file[nume_file])
    #print (text)
    #scegliere quale metodo di suddivisione testo utilizzare
    #
    #frasi_divise = spacy_frasi_divise = divide_in_frase(text)
    #frasi_divise = splitter_result_chunks = custom_text_splitter(text)
    frasi_divise = nltk_sentences = split_text_into_sentences(text)
    # necessario perche su cohere non si possono fare piu di tot chiamate al minuto
    api_call=0
    for frase in frasi_divise:
        api_call+=1
        if api_call<90:
            id_chunk+=1
            response = co.embed(
                    #spacy_frasi_divise
                    #texts=[frase.text],
                    #nltk_sentences
                    texts=[frase],
                    #splitter_result_chunks
                    #texts=[frase non provato non frase[0].page_content
                    model=cohere_model_def,
                    input_type=cohere_input_type
            )
            #print (response)
            vector_result = response.embeddings[0]
            #print (vector_result)
            ##print(id_file,nome_file,vector_result,frase)
            # Inserimento all'interno di oracle vector db  dell'embedding usando bind variable
            cursor.execute("insert into Vector_CV_PDF values (:1,:2,:3,:4,:5)", [id_file,id_chunk,nome_file, vector_result,frase])
        else :
            api_call=0
            # necessario perche su cohere non si possono fare piu di tot chiamate al minuto
            time.sleep(60)
    print ( 'numero di chunk per il pdf  : ' , nome_file, ' -- ',id_chunk)
    connection.commit()

print ( 'numero documenti pdf totali inseriti : ' , id_file)




numero di chunk per il pdf  :  ./docs/CV_PDF/luca_Migliori.pdf  --  15
numero di chunk per il pdf  :  ./docs/CV_PDF/pino_sarsi.pdf  --  16
numero di chunk per il pdf  :  ./docs/CV_PDF/Paolo_checchi.pdf  --  14
numero di chunk per il pdf  :  ./docs/CV_PDF/eugenio_bernaci.pdf  --  16
numero di chunk per il pdf  :  ./docs/CV_PDF/mario_bianchi.pdf  --  13
numero documenti pdf totali inseriti :  5


Verifica contenuto tabella

In [ ]:
# Retrieve the vector
cursor.execute('select * from  Vector_CV_PDF')
#cursor.execute('select max(id_pdf) from  Vector_PDF')
for row in cursor:
     print(row)


Richiesta al modello e conversione in embedding
modificare la domanda a proprio piacimento

In [27]:
#usiamo la chiamata a cohere per trasformare la domanda in vettori 
#usiamo lo stesso modello ma con una tipologia differente orientata alla ricerca
prompt_question = "I've opened a new position for an Italian coding expert in Taleo application , in the meantime could you providing internal name showing me the highlights of this person"
response = co.embed(
        texts=[prompt_question],
        model='embed-english-v3.0',
        input_type='search_query'
)
print(response)

cohere.Embeddings {
	embeddings: [[0.02330017, -0.025817871, -0.027694702, -0.06762695, -0.032440186, -0.02798462, -0.04916382, 0.028549194, 0.026916504, 0.09490967, 0.02458191, -0.015838623, 0.0657959, -0.028564453, -0.05368042, 0.04107666, 0.032684326, 0.033813477, 0.048309326, -0.0021190643, -0.014678955, 0.0446167, 0.015945435, -0.031585693, 0.035064697, -0.04373169, -0.06225586, -0.007511139, 0.029129028, 0.02154541, 0.012321472, 0.02848816, 0.009376526, 0.0015449524, -0.007106781, 0.033569336, -0.008132935, 0.00031352043, 0.006122589, 0.0029411316, 0.010818481, -0.0060157776, -0.026107788, -0.026382446, -0.05557251, -0.0069618225, -0.011116028, -0.0021476746, 0.01675415, -0.07354736, 0.009025574, -0.0146484375, 0.047943115, 0.01777649, -0.023223877, 0.025924683, -0.04348755, -0.035369873, 0.015182495, 0.046905518, 0.025360107, -0.0037841797, 0.008613586, -0.018554688, 0.022460938, -0.0024738312, 0.05895996, 0.004676819, 0.04385376, 0.010482788, -0.039794922, 0.029586792, -0.01751

Ricerca dei chunk nel vector db con il vettore KNN piu vicino 

In [28]:
#Ricerca all'intenro del database oracle sfruttando l'operatore VECTOR_DISTANCE
import array
from typing import List
vector_prompt:List[float]=response.embeddings[0]

cursor = connection.cursor()
sql = """SELECT CHUNK  from Vector_CV_PDF where (ID_PDF, id_chunk) in 
(
select ID_PDF ,substr(ranking,instr(ranking,'-',-1)+1)chunk_id from (
select ID_PDF , min(ranking) ranking
  from (
        select ID_PDF,VECTOR_DISTANCE(V,:1)||'--'||id_chunk ranking 
          from Vector_CV_PDF  
        order by VECTOR_DISTANCE(V,:2)
        )
group by id_pdf order by 2,1 desc FETCH FIRST 3 ROWS ONLY
)
)"""

sql = """SELECT CHUNK  from Vector_CV_PDF where (ID_PDF, id_chunk) in 
(
select ID_PDF ,substr(ranking,instr(ranking,'-',-1)+1)chunk_id from (
select ID_PDF , min(ranking) ranking
  from (
        select ID_PDF,VECTOR_DISTANCE(V,:1)||'--'||id_chunk ranking 
          from Vector_CV_PDF  
        order by VECTOR_DISTANCE(V,:2)
        )
group by id_pdf order by 2,1 desc FETCH FIRST 3 ROWS ONLY
)
)"""

with connection.cursor() as cursor:
        array_query1 = array.array("d", vector_prompt)
        array_query2 = array.array("d", vector_prompt)
        cursor.execute(sql,[array_query1,array_query1])
        rows = cursor.fetchall()
#print(rows)

Funzione cohere per generare testo

In [30]:
# creiamo una funzione per  cohere  per generae la risposta
def generate_text(prompt, temp=0):
  response = co.generate(
    model='command',
    prompt=prompt,
    max_tokens=200,
    temperature=temp)
  return response.generations[0].text

Genera risposta

In [31]:
# richiamiamo la funzione per generare la risposta passando con i chunk ottenuti dal database Oracle

text_prompt ='Kindly answer the following question  ' + prompt_question  + ' based on  ' ;
for i in range(len(rows)):
    text_prompt += rows[i][0]+' and  '

#print (text_prompt)



response = generate_text(text_prompt, temp=0.5)
print(response)

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


Based on the provided information, here are the internal names and role highlights for the three candidates:

1. Paolo Cecchi (Cloud Interconnect Specialist)
   - Internal Name: PCI (Paolo Cecchi Interconnect)
   - Professional Highlights:
     - Experienced Cloud Interconnect Specialist with a proven track record of designing and implementing robust cloud connectivity solutions.
     - Expertise in network architecture and cloud computing, with a focus on providing secure and reliable connections between different cloud platforms.
     - Strong problem-solving and analytical skills, with the ability to find creative solutions to complex technical challenges.
     - Excellent communication and collaboration skills, with a ability to work effectively with cross-functional teams to achieve project goals.

2. Eugenio Bernaci (Cloud Specialist)
   - Internal Name: EB (Eugenio Bernaci)
   - Professional Highlights:
     - Born in 1970 and raised in Rome, with a strong background in cloud co